To run the unzip the files here in "refineSummaryEvents/data", and you should be good to go!



In [11]:
import pandas as pd
import os
import json

folder_path = '../data'

file_paths = []

# Walk through the folder and its subdirectories
for root, dirs, files in os.walk(folder_path):
    for file in files:
        chosen_path = os.path.join(root, file)
        chosen_path.replace('\\', '/')
        file_paths.append(chosen_path)

print(file_paths)

['../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294091_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294102_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294114_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294128_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294129_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294331_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294353_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294369_short.json', '../data\\extracted_game_2023-20231014T022049Z-001\\extracted_game_2023\\ESPORTSTMNT01_3294373_short.json', '../data\\extracted_game_20

In [57]:

json_file_path = file_paths[0]
with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())
nested_df =pd.json_normalize(contents)
nested_df.head()

,eventTime,eventType,platformGameId,gameTime,stageID,participants,gameName,sequenceIndex,teams,gameOver,playbackID
0,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,"[{'ability4CooldownRemaining': 0, 'magicPenetr...",109518697661991477|game1,2918,"[{'inhibKills': 0, 'towerKills': 2, 'teamID': ...",False,1
1,2023-01-21T21:40:44.372Z,stats_update,ESPORTSTMNT01:3294091,1946081,1,"[{'ability4CooldownRemaining': 5.9951171875, '...",109518697661991477|game1,6635,"[{'inhibKills': 1, 'towerKills': 9, 'teamID': ...",True,1


Now we note that we have a list for the participants and some fields in the participants as well as the teams field, so we can play off of this guy (https://www.geeksforgeeks.org/converting-nested-json-structures-to-pandas-dataframes/)

Note however that the teams category requires a "join" operation and using it a basis for merging results in a loss of a lot of data

In [50]:

flattened_via_participants = pd.json_normalize(contents, record_path=["participants"], meta=["eventTime", "eventType", "platformGameId", "gameTime", "stageID", "gameOver", "sequenceIndex", "gameName", "playbackID"])
flattened_via_participants.head()

Index(['ability4CooldownRemaining', 'magicPenetrationPercent', 'alive',
       'participantID', 'spellVamp', 'primaryAbilityResource', 'lifeSteal',
       'primaryAbilityResourceRegen', 'XPForNextLevel',
       'summonerSpell2CooldownRemaining', 'healthMax', 'magicResist',
       'ability3Level', 'summonerSpell1Name', 'teamID', 'currentGold',
       'healthRegen', 'ability4Level', 'level', 'ability1Level',
       'armorPenetration', 'accountID', 'health', 'ability2Level',
       'ultimateName', 'summonerSpell2Name', 'cooldownReduction',
       'magicPenetrationPercentBonus', 'magicPenetration',
       'primaryAbilityResourceMax', 'armorPenetrationPercentBonus',
       'armorPenetrationPercent', 'attackDamage', 'championName',
       'ccReduction', 'shutdownValue', 'playerName', 'respawnTimer',
       'attackSpeed', 'XP', 'ultimateCooldownRemaining', 'armor',
       'stackingBuffs', 'summonerSpell1CooldownRemaining', 'totalGold',
       'items', 'abilityPower', 'ability4Name', 'stats', 

In [48]:
flattened_via_teams = pd.json_normalize(contents, record_path=["teams"], meta=["eventTime", "eventType", "platformGameId", "gameTime", "stageID", "gameOver", "sequenceIndex", "gameName", "playbackID"])
flattened_via_teams.head()

,inhibKills,towerKills,teamID,baronKills,assists,dragonKills,championsKills,totalGold,deaths,eventTime,eventType,platformGameId,gameTime,stageID,gameOver,sequenceIndex,gameName,playbackID
0,0,2,100,0,1,2,2,26431,1,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,False,2918,109518697661991477|game1,1
1,0,0,200,0,1,0,1,22923,2,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,False,2918,109518697661991477|game1,1
2,1,9,100,0,19,4,13,62841,5,2023-01-21T21:40:44.372Z,stats_update,ESPORTSTMNT01:3294091,1946081,1,True,6635,109518697661991477|game1,1
3,0,0,200,0,10,1,5,50459,13,2023-01-21T21:40:44.372Z,stats_update,ESPORTSTMNT01:3294091,1946081,1,True,6635,109518697661991477|game1,1


In [51]:
print(flattened_via_participants.columns, flattened_via_teams.columns)

Index(['ability4CooldownRemaining', 'magicPenetrationPercent', 'alive',
       'participantID', 'spellVamp', 'primaryAbilityResource', 'lifeSteal',
       'primaryAbilityResourceRegen', 'XPForNextLevel',
       'summonerSpell2CooldownRemaining', 'healthMax', 'magicResist',
       'ability3Level', 'summonerSpell1Name', 'teamID', 'currentGold',
       'healthRegen', 'ability4Level', 'level', 'ability1Level',
       'armorPenetration', 'accountID', 'health', 'ability2Level',
       'ultimateName', 'summonerSpell2Name', 'cooldownReduction',
       'magicPenetrationPercentBonus', 'magicPenetration',
       'primaryAbilityResourceMax', 'armorPenetrationPercentBonus',
       'armorPenetrationPercent', 'attackDamage', 'championName',
       'ccReduction', 'shutdownValue', 'playerName', 'respawnTimer',
       'attackSpeed', 'XP', 'ultimateCooldownRemaining', 'armor',
       'stackingBuffs', 'summonerSpell1CooldownRemaining', 'totalGold',
       'items', 'abilityPower', 'ability4Name', 'stats', 

In the spirit of being data greedy ;)

I'll leave it participant centric, for the above reason and also, I feel the whole you're as good as your worst player vibe maybe true, in that variance in skill counts

I just added the team information to each entry for the participant. Questionable decision, but we can always remove the data later in the pipeline. Omitting it now may lead to unnecessary developer pain, when trying to add it in later on.

In [55]:
flattened_with_excessive_data = pd.merge(flattened_via_teams, flattened_via_participants, on="teamID")
flattened_with_excessive_data.head()

,inhibKills,towerKills,teamID,baronKills,assists,dragonKills,championsKills,totalGold_x,deaths,eventTime_x,...,goldStats.shutdownReward,eventTime_y,eventType_y,platformGameId_y,gameTime_y,stageID_y,gameOver_y,sequenceIndex_y,gameName_y,playbackID_y
0,0,2,100,0,1,2,2,26431,1,2023-01-21T21:23:17.283Z,...,NaN,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,False,2918,109518697661991477|game1,1
1,0,2,100,0,1,2,2,26431,1,2023-01-21T21:23:17.283Z,...,NaN,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,False,2918,109518697661991477|game1,1
2,0,2,100,0,1,2,2,26431,1,2023-01-21T21:23:17.283Z,...,NaN,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,False,2918,109518697661991477|game1,1
3,0,2,100,0,1,2,2,26431,1,2023-01-21T21:23:17.283Z,...,NaN,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,False,2918,109518697661991477|game1,1
4,0,2,100,0,1,2,2,26431,1,2023-01-21T21:23:17.283Z,...,NaN,2023-01-21T21:23:17.283Z,stats_update,ESPORTSTMNT01:3294091,899000,1,False,2918,109518697661991477|game1,1


In [56]:
flattened_with_excessive_data.columns

Index(['inhibKills', 'towerKills', 'teamID', 'baronKills', 'assists',
       'dragonKills', 'championsKills', 'totalGold_x', 'deaths', 'eventTime_x',
       'eventType_x', 'platformGameId_x', 'gameTime_x', 'stageID_x',
       'gameOver_x', 'sequenceIndex_x', 'gameName_x', 'playbackID_x',
       'ability4CooldownRemaining', 'magicPenetrationPercent', 'alive',
       'participantID', 'spellVamp', 'primaryAbilityResource', 'lifeSteal',
       'primaryAbilityResourceRegen', 'XPForNextLevel',
       'summonerSpell2CooldownRemaining', 'healthMax', 'magicResist',
       'ability3Level', 'summonerSpell1Name', 'currentGold', 'healthRegen',
       'ability4Level', 'level', 'ability1Level', 'armorPenetration',
       'accountID', 'health', 'ability2Level', 'ultimateName',
       'summonerSpell2Name', 'cooldownReduction',
       'magicPenetrationPercentBonus', 'magicPenetration',
       'primaryAbilityResourceMax', 'armorPenetrationPercentBonus',
       'armorPenetrationPercent', 'attackDamage', '